## Collaborative Filtering Approach

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

#### About Books dataset

In [2]:
books=pd.read_csv('/kaggle/input/bookcrossing/BX-Books.csv', sep=";",on_bad_lines='skip' , encoding='latin-1')

/tmp/ipykernel_42/497011900.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv('/kaggle/input/bookcrossing/BX-Books.csv', sep=";",on_bad_lines='skip' , encoding='latin-1')


In [3]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [4]:
books.shape

(271360, 8)

In [5]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [6]:
books=books[['ISBN','Book-Title','Book-Author','Year-Of-Publication', 'Publisher','Image-URL-L']]

In [7]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


In [8]:
books.rename(columns={
    "Book-Title":'title',
    "Book-Author":'author',
    "Year-Of-Publication":"year",
    "Publisher":"publisher",
    "Image-URL-L":"img_url"},inplace=True )

In [9]:
books.head(2)

,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...


#### About Users Dataset

In [10]:
user=pd.read_csv('/kaggle/input/bookcrossing/BX-Users.csv', sep=";", encoding='latin-1',on_bad_lines='skip')

In [11]:
user.head(2)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [12]:
user.shape

(278858, 3)

In [13]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


#### Importing ratings dataset

In [16]:
ratings=pd.read_csv('/kaggle/input/bookcrossing/BX-Book-Ratings.csv', sep=";", encoding='latin-1',on_bad_lines='skip')

In [17]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [18]:
ratings.shape

(1149780, 3)

In [19]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


**In collaborative filtering recommendation systems, ratings are most neccessary to perform it.**

In [21]:
print(books.shape, user.shape, ratings.shape, sep='\n')

(271360, 6)
(1149780, 3)
(1149780, 3)


In [26]:
ratings.rename(columns={
    "User-ID":"user_id",
    "Book-Rating":"ratings"
}, inplace=True)

In [27]:
ratings.head(2)

,user_id,ISBN,ratings
0,276725,034545104X,0
1,276726,0155061224,5


In [29]:
ratings['user_id'].value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

- As you can see here are multiple users rated the book and if you look on the tail side, all you can say that 1 review per book is not really helpfull.
- So we have to make some change in here.
-**So I am applying a filter here, those who have liked the book or rated the book more than 200.**

In [30]:
# number of unique users 
ratings['user_id'].value_counts().shape

(105283,)

In [31]:
ratings['user_id'].unique().shape

(105283,)

**Filtering**

In [32]:
x=ratings['user_id'].value_counts() > 200

In [33]:
# The users who have rated 200 more ratings
x

user_id
11676      True
198711     True
153662     True
98391      True
35859      True
          ...  
116180    False
116166    False
116154    False
116137    False
276723    False
Name: count, Length: 105283, dtype: bool

In [34]:
x[x].shape

(899,)

**only 899 users rated more than 200 books.**

### Index of the users who rated 200 or more than ratings.

In [35]:
y=x[x].index

In [36]:
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [38]:
ratings=ratings[ratings['user_id'].isin(y)]

In [39]:
ratings.head()

,user_id,ISBN,ratings
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [40]:
ratings.shape

(526356, 3)

#### Merging ratings to books dataset
 - We are merging using ISBN columns, because it is same column present in both dataset.

In [41]:
ratings_with_books=ratings.merge(books, on='ISBN')

In [43]:
ratings_with_books.head()

,user_id,ISBN,ratings,title,author,year,publisher,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [44]:
ratings_with_books.shape

(487671, 8)

In [52]:
num_rating=ratings_with_books.groupby('title')['ratings'].count().reset_index()

In [53]:
num_rating.head()

,title,ratings
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [55]:
num_rating.shape

(160269, 2)

In [56]:
num_rating.rename(columns={
    'ratings':'num_of_rating'
}, inplace=True)

In [57]:
num_rating.head()

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


**Now we merge again the title column with ratings_with_books dataset.**

In [59]:
final_rating=ratings_with_books.merge(num_rating, on='title')

In [60]:
final_rating.head()

,user_id,ISBN,ratings,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


In [61]:
final_rating.shape

(487671, 9)

- Now we will filterted again
- Those which books are rated 50, we would like to consider here.

In [63]:
final_rating=final_rating[final_rating['num_of_rating']>=50]

In [66]:
# As you can see all are rated 50 or more than 50
final_rating.sample(10)

,user_id,ISBN,ratings,title,author,year,publisher,img_url,num_of_rating
109805,268330,0451197410,0,How Stella Got Her Groove Back,Terry McMillan,1998,Signet Book,http://images.amazon.com/images/P/0451197410.0...,101
1180,156150,0142001740,0,The Secret Life of Bees,Sue Monk Kidd,2003,Penguin Books,http://images.amazon.com/images/P/0142001740.0...,209
9052,107453,0451169522,0,Misery,Stephen King,2004,Penguin USA (Paper),http://images.amazon.com/images/P/0451169522.0...,121
8590,144531,0446525774,0,Saving Faith,David Baldacci,1999,Warner Books,http://images.amazon.com/images/P/0446525774.0...,86
107810,107301,0375703861,0,White Teeth: A Novel,Zadie Smith,2001,Vintage Books USA,http://images.amazon.com/images/P/0375703861.0...,52
35431,150498,0440226430,0,Summer Sisters,Judy Blume,1999,Dell Publishing Company,http://images.amazon.com/images/P/0440226430.0...,188
89896,217375,038550926X,0,The Devil Wears Prada : A Novel,LAUREN WEISBERGER,2003,Doubleday,http://images.amazon.com/images/P/038550926X.0...,75
123815,52584,0671822209,0,WLD ACCORDNG GARP,John Irving,1979,Pocket,http://images.amazon.com/images/P/0671822209.0...,54
19839,260897,0440222656,0,The Horse Whisperer,Nicholas Evans,1996,Dell,http://images.amazon.com/images/P/0440222656.0...,213
56224,110483,051513287X,0,Face the Fire (Three Sisters Island Trilogy),Nora Roberts,2002,Jove Books,http://images.amazon.com/images/P/051513287X.0...,99


In [65]:
final_rating.shape

(61853, 9)

In [68]:
# Checking null values
final_rating.isnull().sum()

user_id          0
ISBN             0
ratings          0
title            0
author           0
year             0
publisher        0
img_url          0
num_of_rating    0
dtype: int64

**Drop some duplicates where the title name and book name must be same.**

In [70]:
final_rating.drop_duplicates(['user_id','title'], inplace=True)

In [71]:
final_rating.shape

(59850, 9)

### Our final dataset is here 

In [72]:
final_rating.head()

,user_id,ISBN,ratings,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82


#### Lets create a pivot table to make similar users and then cluster them.

In [74]:
book_pivot=final_rating.pivot_table(columns='user_id', index='title', values='ratings')

In [75]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [76]:
book_pivot.shape

(742, 888)

**Replacing Nan values with 0**

In [77]:
book_pivot.fillna(0, inplace=True)

In [78]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Train Test Split

### There is a problem here

- Clustering means distance, but in our dataset there are so much zero enetities and if we perfoming the distnace calculation it takes a long time.

#### So we used here csr_matrix to solve the issue :-
1. **we use csr matrix_ to not consider the zeros while calculating.**
2. *It will ignore the zeros*

In [80]:
from scipy.sparse import csr_matrix

In [81]:
book_sparse=csr_matrix(book_pivot)

In [82]:
book_sparse

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14942 stored elements in Compressed Sparse Row format>

### Implementing Clustering algorithm

In [85]:
from sklearn.neighbors import NearestNeighbors

**We are using brute algorithm here to identify the clusters.**

In [87]:
model=NearestNeighbors(algorithm='brute')

In [88]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

**n_neighbors means number of recommendation are we want to show.**

In [90]:
distance, suggestion=model.kneighbors(book_pivot.iloc[237, :].values.reshape(1,-1), n_neighbors=6)

In [94]:
# zero means with itself
# all six books recommendings
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [93]:
suggestion

array([[237, 240, 238, 241, 184, 536]])

**To see the name of the books**

In [95]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])
    # 237 index is Harry Poter book

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


#### To show all the book titles

In [97]:
book_pivot.index

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [98]:
book_name=book_pivot.index

### Saving Models

In [103]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))
pickle.dump(book_name, open('book_name.pkl', 'wb'))

# To get the posters
pickle.dump(final_rating, open('final_rating.pkl', 'wb'))

pickle._dump(book_pivot, open('book_pivot.pkl', 'wb'))